In [24]:
import zarr
import pysam
import numpy as np

In [25]:
m = np.array([1,2,4])
n = np.array([1,3,4])

np.stack((n,m), axis=0)

array([[1, 3, 4],
       [1, 2, 4]])

In [26]:
store = zarr.storage.MemoryStore()
z = zarr.create_array(store=store, shape=(10, 0), chunks=(10, 100), dtype='int32')
z.shape



(10, 0)

In [27]:
m = np.arange(200).reshape((10,20))
z.append(m, axis = 1)
z.shape

(10, 20)

In [28]:

z2 = zarr.create_array(zarr.storage.MemoryStore(), shape=(1,0), chunks=(1,10), dtype='int32')
z2.append(np.array([1,2]).reshape(1,-1), axis=1)
z2[:,:]

array([[1, 2]], dtype=int32)

In [29]:
z[:,:]

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19],
       [ 20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
         33,  34,  35,  36,  37,  38,  39],
       [ 40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59],
       [ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
         73,  74,  75,  76,  77,  78,  79],
       [ 80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
         93,  94,  95,  96,  97,  98,  99],
       [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
        113, 114, 115, 116, 117, 118, 119],
       [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
        133, 134, 135, 136, 137, 138, 139],
       [140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
        153, 154, 155, 156, 157, 158, 159],
       [160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 1

In [30]:
da1 = zarr.open_group('../data/01_processed/ssl_sets/da1_subset_10k.zarr')

In [36]:
read=50
da1['data'][:,int(da1['indptr'][read]):int(da1['indptr'][read+1])]

KeyError: 'data'

In [32]:
da1['indptr'][1]


array(20986, dtype=uint32)

In [34]:
da1['data'].shape

(6, 171219240)

In [35]:
171219240/10_000

17121.924

In [5]:
x = [1,2] + [2,3]
print(x)

[1, 2, 2, 3]
